In [ ]:
# COVID-19 mask detector - https://github.com/alexcamargoweb/mask-detector
# Detecção de máscaras em pessoas (com ou sem)
# Adrian Rosebrock, COVID-19: Face Mask Detector with OpenCV, Keras/TensorFlow, and Deep Learning. PyImageSearch.    
# Disponível em: https://www.pyimagesearch.com/2018/09/24/opencv-face-recognition/.   
# Acessado em: 05/12/2020.  
# Arquivo: predict_mask-image.ipynb
# Execução via Google Colab

In [ ]:
# monta o google drive
from google.colab import drive, output, files
from google.colab.patches import cv2_imshow
drive.mount('/content/drive')
# acessa o diretório do projeto
%cd /content/drive/MyDrive/mask-detection
# limpa as saídas
output.clear()

# PACOTES UTILIZADOS

# importa os pacotes necessários
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os

In [ ]:
# PARÂMETROS DE EXECUÇÃO

# carrega o detector
DETECTOR = '/content/drive/MyDrive/detector'
# carrega o modelo
MODEL = '/content/drive/MyDrive/models/mask_covid-19.model'
# confiança de detecção
CONFIDENCE = 0.5

# INICIA A DETECÇÃO

# carrega o detector de faces
print("[INFO] carregando detector de faces...")
# estrutura da rede
prototxtPath = os.path.sep.join([DETECTOR, "deploy.prototxt"])
# pesos da rede
weightsPath = os.path.sep.join([DETECTOR, "res10_300x300_ssd_iter_140000.caffemodel"]) 
net = cv2.dnn.readNet(prototxtPath, weightsPath)
# carrega do disco o modelo detector de máscaras
print("[INFO] carregando detector de faces...")
model = load_model(MODEL)

# remove as imagens preditas anteriormente (se houverem)
!rm -R /content/drive/MyDrive/inputs/* > /dev/null 
# entra no diretório de inputs de imagens
%cd /content/drive/MyDrive/inputs
output.clear()
# permite o upload de uma imagem (pessoa com ou sem máscara)
file_ = files.upload() 
# guarda o caminho da imagem enviada
DIR = '/content/drive/MyDrive/inputs'
# guarda o nome da imagem
IMG = list(file_)[0] 

# carrega a imagem
image = cv2.imread(DIR+'/'+IMG)
# faz uma cópia
orig = image.copy()
# pega as dimenções
(h, w) = image.shape[:2]
# constrói um blob (objeto do tipo arquivo) da imagem
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))

# passa o blob para a rede e obtém a detecção da face
print("[INFO] processando detecção de face...")
net.setInput(blob)
detections = net.forward()

# faz um loop sobre as detecções
for i in range(0, detections.shape[2]):
	# extrai a confiança (por exemplo, a probabilidade) assossiada à detecção
	confidence = detections[0, 0, i, 2]
	# filtra as detecções fracas, garantindo que a confiança é
	# maior do que a confiança mínima
	if confidence > CONFIDENCE:
		# processa as coordenadas x e y da bouding box do objeto
		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")
		# garante que as bouding boxes estejam dentro das dimensões do frame
		(startX, startY) = (max(0, startX), max(0, startY))
		(endX, endY) = (min(w - 1, endX), min(h - 1, endY))
	
		# extrai o ROI do rosto, converte- do canal BGR para RGB
		# redimensiono para 224x224 e realiza o pré-processamento
		face = image[startY:endY, startX:endX]
		face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
		face = cv2.resize(face, (224, 224))
		face = img_to_array(face)
		face = preprocess_input(face)
		face = np.expand_dims(face, axis=0)
		# passa o rosto pelo modelo para determinar se
		# tem uma máscara ou não
		(mask, withoutMask) = model.predict(face)[0]

		# determina o label da classe e a cor para desenhar
		# a bouding box e o texto
		label = "Com mascara" if mask > withoutMask else "Sem mascara"
		color = (0, 255, 0) if label == "Com mascara" else (0, 0, 255)
		# inclui a probabilidade no label
		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
		# exibe o label e o retângulo da bounding box no frame de saída
		cv2.putText(image, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)

# exibe a imagem de saída
cv2_imshow(image)
cv2.waitKey(0)